In [ ]:
import numpy as np
import scipy.io as scio
import sys
sys.path.append('/Users/celia/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/celia/GitHub/mouse_bandit')
import pandas as pd
import bandit_preprocessing as bp
import sys
import os

In [16]:
# set model traces to its own variable

nNeurons = neuron.C.shape[0] #number of neurons

nan_neurons = []
for iNeuron in range(nNeurons):
    if any(np.isnan(neuron.C_raw[iNeuron,:])) == True:
        nan_neurons.append(iNeuron)
good_neurons = [x for x in range(0, nNeurons) if x not in nan_neurons]

In [25]:
# remove neurons with NaNs from relevant variables
neuron.C = neuron.C[good_neurons, :]
neuron.C_raw = neuron.C_raw[good_neurons,:]

# refined count for neurons after cleansing NaNs
nNeurons = neuron.C.shape[0]

"""
set up system to detect events:
first detect any events above a threshold
then make sure the events are a least a minimum duration?
only mark events at the beginning
"""

#Create Gaussian filter and apply to raw trace
sigma = 1.5;
sz = 5;  

x = np.linspace(-sz / 2, sz / 2, sz);
gaussFilter = np.exp(-x**2 / (2*sigma**2));
gaussFilter = gaussFilter / np.sum(gaussFilter);

smoothed = np.zeros((nNeurons, neuron.C_raw.shape[1]+sz-1));

for i in range(0, nNeurons):
    smoothed[i,:] = np.convolve(neuron.C_raw[i,:], gaussFilter);

"""
Z-score neurons and set threshold for events. Shift by 1 and subtract to produce 0s and 1s.
"""

z_neuron = np.zeros((nNeurons, neuron.C_raw.shape[1]))
for i in range(0,nNeurons):
    z_neuron[i,:] = (neuron.C_raw[i,:] - np.mean(neuron.C_raw[i,:], axis=0)) / np.std(neuron.C_raw[i,:], axis=0)
thresh = 5.
thresh_neuron = z_neuron > thresh

thresh_shift = np.insert(thresh_neuron, 0, 0 , axis=1)
thresh_shift = thresh_shift[:,0:thresh_shift.shape[1]-1]

"""
Remove timepoints of decay so events only mark onset times
"""

events_on_off = thresh_neuron - thresh_shift
events = events_on_off 

for iNeuron in range(0,nNeurons):
    indices = np.nonzero(events[iNeuron,:])
for ind in range(0,np.size(indices)):
    if smoothed[iNeuron,indices[0][ind]] - smoothed[iNeuron,(indices[0][ind]-1)]<0:
        events[iNeuron,indices[0][ind]] = 0
        indices[0][ind] = 0
    if ind>0:
        if indices[0][ind]-4 <= indices[0][ind-1]:
            events[iNeuron, indices[0][ind]] = 0
#indices[0][ind] = 0

/Users/celia/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide
/Users/celia/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in greater
/Users/celia/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.


In [30]:
np.sum(smoothed)

0.0